In [30]:
import tensorflow as tf
from sklearn.utils import shuffle
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

class DNN(object):
    def __init__(self, n_in, n_hiddens, n_out):
        # 초기화처리
        self.n_in = n_in
        self.n_hiddens = n_hiddens
        self.n_out = n_out
        self.weights = []
        self.biases = []
        
        self._x = None
        self._t = None
        self._keep_prob = None
        self._sess = None
        self._history = {
            'accuracy': [], 'loss': []
        }
    
    def weight_variable(self, shape):
        initial = tf.truncated_normal(shape, stddev=0.01)
        return tf.Variable(initial)
    
    def bias_variable(self, shape):
        initial = tf.zeros(shape)
        return tf.Variable(initial)
    
    def inference(self, x, keep_prob):
        # 모델 정의
        for i, n_hidden in enumerate(self.n_hiddens):
            if i == 0:
                input = x
                input_dim = self.n_in
            else:
                input = output
                input_dim = self.n_hiddens[i-1]
            
            self.weights.append(self.weight_variable([input_dim, n_hidden]))
            self.biases.append(self.bias_variable([n_hidden]))
            
            h = tf.nn.relu(tf.matmul(input, self.weights[-1]) + self.biases[-1])
            output = tf.nn.dropout(h, keep_prob)
        
        self.weights.append(self.weight_variable([self.n_hiddens[-1], self.n_out]))
        self.biases.append(self.bias_variable([self.n_out]))
        
        y = tf.nn.softmax(tf.matmul(output, self.weights[-1]) + self.biases[-1])
        return y
    
    def loss(self, y, t):
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(t*tf.log(tf.clip_by_value(y, 1e-10, 1.0)), reduction_indices=[1]))
        return cross_entropy
    
    def training(self, loss):
        optimizer = tf.train.GradientDescentOptimizer(0.01)
        train_step = optimizer.minimize(loss)
        return train_step
    
    def accuracy(self, y, t):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(t, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        return accuracy
    
    def fit(self, X_train, y_train, epochs=100, batch_size=100, p_keep=0.5, verbose=1):
        # 학습 처리
        x = tf.placeholder(tf.float32, shape=[None, self.n_in])
        t = tf.placeholder(tf.float32, shape=[None, self.n_out])
        keep_prob = tf.placeholder(tf.float32)
        
        # evaluate() 용으로 작성해두기
        self._x = x
        self._t = t
        self._keep_porb = keep_prob
        
        y = self.inference(x, keep_prob)
        loss = self.loss(y, t)
        train_step = self.training(loss)
        accuracy = self.accuracy(y, t)
        
        init = tf.global_variables_initializer()
        sess = tf.Session()
        sess.run(init)
        
        # evaluate() 용
        self._sess = sess
        
        N_train = len(X_train)
        n_batches = N_train // batch_size
        
        for epoch in range(epochs):
            X_, Y_ = shuffle(X_train, y_train)
            
            for i in range(n_batches):
                start = i*batch_size
                end = start + batch_size
                
                sess.run(train_step, feed_dict={
                    x:X_train, t:y_train, keep_prob:p_keep
                })
                
            loss_ = loss.eval(session=sess, feed_dict={
                x:X_train, t:y_train, keep_prob:1.0
            })
            
            accuracy_ = accuracy.eval(session=sess, feed_dict={
                x:X_train, t:y_train, keep_prob:1.0
            })

            self._history['loss'].append('loss_')
            self._history['accuracy'].append('accuracy_')
            
            if verbose:
                print('epoch:', epoch,
                     'loss:', loss_,
                     'accuracy:', accuracy_)
                
        return self._history
    
    def evaluate(self, X_test, y_test):
        # 평가처리
        return self.accuracy.eval(session=self._sess, feed_dict={
            self._x:X_test, self._t:y_test, self._keep_prob: 1.0
        })

In [31]:
from keras.datasets import mnist
import numpy as np
(X_train, y_train), (X_test, y_test) = mnist.load_data()
y_train, y_test = np.eye(10)[y_train.astype(int)], np.eye(10)[y_test.astype(int)]
X_train = X_train.reshape(-1, 28*28)
X_train.shape
X_test = X_test.reshape(-1, 28*28)
X_test.shape
X_train, X_test = X_train.astype(np.float64), X_test.astype(np.float64)
n_in = len(X_train[0])
n_in
n_hidden = 200
n_out = 10

In [32]:
model = DNN(n_in=784, n_hiddens=[200, 200, 200], n_out=10)

In [33]:
# GPU 사용해보기
# import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

with tf.device("/device:GPU:0"):
    model.fit(X_train, y_train, epochs=10, batch_size=200, p_keep=0.5)
    
# epoch 3부터 학습이 아예 안되는 것 확인 가능 => 경사소실 문제
# class 내에서 loss function 에 clip_by_value로 수정하기 

epoch: 0 loss: 0.40084717 accuracy: 0.88585
epoch: 1 loss: 0.22031508 accuracy: 0.9361
epoch: 2 loss: 0.15581676 accuracy: 0.9547667
epoch: 3 loss: 0.12221841 accuracy: 0.96465
epoch: 4 loss: 0.10068779 accuracy: 0.97108334
epoch: 5 loss: 0.08509092 accuracy: 0.9761
epoch: 6 loss: 0.073139 accuracy: 0.97941667
epoch: 7 loss: 0.06360354 accuracy: 0.9821333
epoch: 8 loss: 0.0559769 accuracy: 0.9844
epoch: 9 loss: 0.049479917 accuracy: 0.9860333


### 이번에도 Keras로도 해보자

In [35]:
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import SGD
from keras.layers.core import Dropout

n_in = 784
n_hiddens = [200, 200, 200]
n_out = 10
p_keep = 0.5
activation = 'relu'

model = Sequential()
for i, input_dim in enumerate(([n_in]+n_hiddens)[:-1]):
    model.add(Dense(n_hiddens[i], input_dim=input_dim))
    model.add(Activation(activation))
    model.add(Dropout(p_keep))
model.add(Dense(n_out))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.01), metrics=['accuracy'])
epochs = 50
batch_size = 200

hist = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))
# 학습이 잘 안됨... => 웨이트 초기화 기법을 사용해야 함

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
59600/60000 [============================>.] - ETA: 0s - loss: 8.0375 - acc: 0.1424

C:\Users\Main\Anaconda3\lib\site-packages\keras\engine\training_v1.py:2045: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


60000/60000 [==============================] - 2s 33us/sample - loss: 7.9974 - acc: 0.1429 - val_loss: 1.7773 - val_acc: 0.3534
Epoch 2/50
60000/60000 [==============================] - 2s 28us/sample - loss: 2.0056 - acc: 0.2502 - val_loss: 1.6467 - val_acc: 0.4225
Epoch 3/50
60000/60000 [==============================] - 2s 28us/sample - loss: 1.8944 - acc: 0.3023 - val_loss: 1.5432 - val_acc: 0.4538
Epoch 4/50
60000/60000 [==============================] - 2s 28us/sample - loss: 1.8259 - acc: 0.3243 - val_loss: 1.4488 - val_acc: 0.4883
Epoch 5/50
60000/60000 [==============================] - 2s 29us/sample - loss: 1.7725 - acc: 0.3428 - val_loss: 1.4118 - val_acc: 0.5185
Epoch 6/50
60000/60000 [==============================] - 2s 29us/sample - loss: 1.7215 - acc: 0.3603 - val_loss: 1.3152 - val_acc: 0.5474
Epoch 7/50
60000/60000 [==============================] - 2s 30us/sample - loss: 1.7030 - acc: 0.3674 - val_loss: 1.3641 - val_acc: 0.5304
Epoch 8/50
60000/60000 [==============

In [41]:
from keras.initializers import TruncatedNormal

model = Sequential()
for i, input_dim in enumerate(([n_in]+n_hiddens)[:-1]):
    model.add(Dense(n_hiddens[i], input_dim=input_dim, kernel_initializer=TruncatedNormal(stddev=0.01)))
    model.add(Activation(activation))
    model.add(Dropout(p_keep))
model.add(Dense(n_out, kernel_initializer=TruncatedNormal(stddev=0.01)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer=SGD(learning_rate=0.01), metrics=['accuracy'])
epochs = 50
batch_size = 200
hist = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test))

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 2s 32us/sample - loss: 1.3713 - acc: 0.5203 - val_loss: 0.4297 - val_acc: 0.8681
Epoch 2/50
60000/60000 [==============================] - 2s 28us/sample - loss: 0.4661 - acc: 0.8617 - val_loss: 0.2228 - val_acc: 0.9345
Epoch 3/50
60000/60000 [==============================] - 2s 28us/sample - loss: 0.3364 - acc: 0.9033 - val_loss: 0.1791 - val_acc: 0.9475
Epoch 4/50
60000/60000 [==============================] - 2s 28us/sample - loss: 0.2760 - acc: 0.9212 - val_loss: 0.1469 - val_acc: 0.9555
Epoch 5/50
60000/60000 [==============================] - 2s 27us/sample - loss: 0.2394 - acc: 0.9314 - val_loss: 0.1347 - val_acc: 0.9597
Epoch 6/50
60000/60000 [==============================] - 2s 27us/sample - loss: 0.2175 - acc: 0.9375 - val_loss: 0.1226 - val_acc: 0.9636
Epoch 7/50
60000/60000 [==============================] - 2s 27us/sample - loss: 0.2016 - acc: 0.9432 - val_loss: 0.